In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install scikit-learn==0.24

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
# regression model
from sklearn.ensemble import RandomForestRegressor
# metrics packages
from sklearn.metrics import make_scorer, mean_absolute_percentage_error, mean_squared_error


# functions

In [ ]:
def hyper_parameter_search(data, X, baseline_start, baseline_end, regressor):
  # get date range for baseline and select data
  time_base = pd.date_range(start=baseline_start, end=baseline_end, freq='2MS')
  data_base = data[time_base]

  # get climate data for baseline and stressor
  X_base = X.loc[time_base]

  # initialize best params matrix
  best_params = pd.DataFrame(index=data_base.index, columns=['params','best_score', 'mape', 'rmse'])

  # normalize data
  X_base_scaled = scaler.fit_transform(X_base)

  # predict consumption during the stressor for each account by Random Forrest Regressor
  for i, row in data_base.iterrows():
    y = row.array
    model = regressor.fit(X_base_scaled, y)
    y_pred = model.predict(X_base_scaled)
    mape = mean_absolute_percentage_error(y_true=y, y_pred=y_pred)
    rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)

    best_params.loc[i] = (model.best_params_, model.best_score_,mape, rmse)

  return best_params

In [ ]:
def run_model(data, climate_data, baseline_start, baseline_end,  best_params):
  # get date range for baseline and select data
  time_base = pd.date_range(start=baseline_start, end=baseline_end, freq='2MS')
  data_base = data[time_base]

  # get climate data for baseline and stressor
  X_base = climate_data.loc[time_base]
  X_base_scaled = scaler.fit_transform(X_base)
 

  # initialize quantile predictions
  quantiles = [0.1, 0.2, 0.5, 0.8, 0.9]
  prediction_rf = pd.DataFrame(index=data_base.index, columns=['params', 'q0.1','q0.2', 'q0.5', 'q0.8', 'q0.9', 'y_true', 'date'])

  # predict consumption during the stressor for each account by Random Forrest Regressor
  for i, row in data_base.iterrows():
    print(i)
    y = row.array
    params = best_params['params'].loc[i]
    regressor = RandomForestRegressor(**params)
    regressor.fit(X_base_scaled, y)
    preds = [rf_quantile(regressor, X_base_scaled, q) for q in quantiles]
    prediction_rf.loc[i] = (params, preds[0], preds[1], preds[2], preds[3], preds[4], y.to_numpy(), time_base)


  return prediction_rf

In [ ]:
def rf_quantile(m, X, q):
    rf_preds = []
    for estimator in m.estimators_:
        rf_preds.append(estimator.predict(X))
    rf_preds = np.array(rf_preds).transpose()  # One row per record.
    return np.percentile(rf_preds, q * 100, axis=1)

# data loading and preprocessing


In [ ]:
# define customer group
customers = 'MD'

In [ ]:
# load consumption data, climatic and employment features
target_data = pd.read_pickle('/content/drive/MyDrive/Stanford-TUBerlin/CodePaper/Consumption_Matrices/{}_accounts_actual_usage'.format(customers))
features = pd.read_csv('/content/drive/MyDrive/Stanford-TUBerlin/CodePaper/climate_economic_features' , index_col=0)

In [ ]:
# process features
features.index= pd.to_datetime(features.index)
features['month'] = features.index.month_name()
# create binary variable for month
X = pd.get_dummies(features, columns = ['month'])

In [ ]:
target_data

# Cross-Validation

In [ ]:
#### random grid ####
# Number of trees in random forest
n_estimators = [50, 100, 150, 250, 500]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth =[]# [int(x) for x in np.linspace(10, 100, num = 8)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

{'n_estimators': [50, 100, 150, 250, 500], 'max_features': ['auto', 'sqrt'], 'max_depth': [None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}


In [ ]:
# Use the random grid to search for best hyperparameters
# custom scoring
scoring = {'MAPE': make_scorer(mean_absolute_percentage_error)}#, 'MSE': make_scorer(mean_squared_error)}
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3,verbose=2,  random_state=142, n_jobs = -1)#, scoring=make_scorer(mean_absolute_percentage_error))

# initialize scaler to normalize features
scaler = MinMaxScaler()

In [ ]:
# start cross validation

# shuffel customers and select 100 
#data_100 = target_data.sample(n=10, random_state=256354)

#best_params_100 = hyper_parameter_search(data_100, X, '1/1/2002', '11/1/2007', rf_random)

In [ ]:
#best_params_100.mape.mean()

In [ ]:
# start hyperparameter search with cross validation
# divide data into sets of 500 customers

best_params_MD = hyper_parameter_search(target_data, X, '1/1/2002', '11/1/2007', rf_random)
best_params_MD.to_pickle('/content/drive/MyDrive/Stanford-TUBerlin/CodePaper/Hyperparams_MD/best_params_MD')